<a href="https://colab.research.google.com/github/shin0105/4YP/blob/master/optimization_problems_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import jaxopt
from tinygp import GaussianProcess, kernels, transforms
jax.config.update("jax_enable_x64", True)

#BOHACHEVSKY FUNCTIONS using tinygp (Matern series)

#BOHACHEVSKY FUNCTIONS description
random = np.random.default_rng()
X = random.uniform(-100, 100, (100, 2))
yerr = 0.01
y = X[:,0]**2 + 2*X[:,1]**2 - 0.3*np.cos(3*np.pi*X[:,0]) - 0.4*np.cos(4*np.pi*X[:,1]) + 0.7 + yerr * random.normal(
    size=len(X))

X1,X2 = np.meshgrid(X[:,0], X[:,1])
y_true = X1**2 + 2*X2**2 - 0.3*np.cos(3*np.pi*X1) - 0.4*np.cos(4*np.pi*X2) +0.7
X_pred = np.vstack((X1.flatten(), X2.flatten())).T

def train_gp(nparams, build_gp_func):
    @jax.jit
    def loss(params):
        return -build_gp_func(params).log_probability(y)

    params = {
        "log_amp": np.float64(0.0),
        "log_scale": np.zeros(nparams),
    }
    solver = jaxopt.ScipyMinimize(fun=loss)
    soln = solver.run(params)
    return build_gp_func(soln.params)

#Kernel = Matern series v=3/2, uncorrelated 
def build_gp_uncorr(params):
    kernel = jnp.exp(params["log_amp"]) * transforms.Linear(
        jnp.exp(-params["log_scale"]), kernels.Matern32()
    )
    return GaussianProcess(kernel, X, diag=yerr**2)

uncorr_gp = train_gp(2, build_gp_uncorr)

y_pred = uncorr_gp.condition(y, X_pred).gp.loc.reshape(y_true.shape)

#printing results, comparing trained(predicted) data vs. true value
print(f"Predicted minimum y value using GP: {y_pred.min()}")

print(f"Minimum y value from sampled points: {y_true.min()}")

Predicted minimum y value using GP: 2.6704434607127935e-22
Minimum y value from sampled points: 1.0943808026554311
